In [65]:
import numpy as np
import cupy as cp
import pandas as pd
import cudf
df = pd.read_csv('Iris.csv')


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from cuml.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score as bas

df = df.drop('Species', axis=1)
y=df.Target
X_train, X_test, y_train, y_test = train_test_split(df, y,test_size=0.2)
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

P = bas(y_test, y_pred)
print(P)

y_test = cp.asarray(y_test) 
y_pred = cp.asarray(y_pred)


M = confusion_matrix(y_test, y_pred, labels=None,
                     sample_weight=None,
                     normalize=None)

classes = cp.unique(y_test)
def balanced_accuracy_score(conf_matrix, classes, adjusted=False):
    rec_list = cp.empty(len(classes))
    for i in range(len(classes)): 
        TP = conf_matrix[i,i]
        D = 0
        for j in range(0, len(classes)):
                D = D + conf_matrix[i, j] #TP+FN
        try:
          rec = TP/D
          rec_list[i] = rec
        except ZeroDivisionError:
          rec_list[i] = 0
    if cp.any(cp.isnan(rec_list)):
        warnings.warn('y_pred contains classes not in y_true')
        rec_list = rec_list[~cp.isnan(rec_list)]
    balanced_score = cp.sum(rec_list)/len(classes)
    if adjusted:
        n_classes = len(rec_list) 
        chance = 1 / n_classes 
        balanced_score -= chance 
        balanced_score /= 1 - chance
    return balanced_score
balanced_accuracy_score(M, classes)




1.0


/opt/conda-environments/rapids-stable/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array(1.)